# Financial News Analysis Research Assistant

## Multi-Agent System for Real-Time Financial Analysis

### Project Overview:

This system implements a sophisticated multi-agent architecture designed to analyze financial news in real-time and provide intelligent, context-aware responses to user queries. The pipeline integrates multiple specialized agents that work collaboratively to retrieve, analyze, and synthesize financial information.

### Core Architecture:

- **News Retrieval Agent**: Fetches real-time financial news from multiple sources

- **Sentiment Analysis Agent**: Performs sentiment scoring on news content

- **Topic Classification Agent**: Routes queries to appropriate domain specialists

- **Specialist Agents**: Domain-specific models fine-tuned for financial topics

- **Evaluation & Optimization**: Continuous improvement through feedback loops

- **Memory System**: Maintains conversation history and context

**Technical Stack**: Python, HuggingFace Transformers, PyTorch, Custom Fine-tuned Models

### 1. System Initialization & Agent Imports

*This section imports all specialized agents that form our multi-agent architecture, each responsible for a specific task in the financial analysis pipeline.*

In [15]:
import sys
import os
sys.path.append(os.path.abspath('src'))

from utils.ticker_finder import get_ticker_from_company_name
from memory.memory_agent import MemoryAgent
from agents.topic_classifier_agent import TopicClassifier
from agents.news_retrieval_agent import NewsRetrievalAgent
from agents.sentiment_analysis_agent import SentimentAnalysisAgent
from agents.specialist_agent import Specialist
from agents.evaluator_optimizer_agent import EvaluatorOptimizer

### 2. User Query Processing

**Objective**: Process natural language financial queries from users. In production, this would come from an API endpoint or user interface.

In [16]:
query = "How are apple's stocks doing?"

### 3. Company Identification & Historical Context

**Function**: Extracts company names from user queries and converts them to stock ticker symbols for accurate financial data retrieval.

**Purpose**: Maintains conversation context by storing and retrieving past interactions, enabling the system to provide consistent, personalized responses.

In [22]:
companies = ["apple", "microsoft", "google", "tesla", "nvidia", "intel"]
for company in companies:
    if company in query.lower():
        company_name = company

ticker = get_ticker_from_company_name(company_name)

memory_agent = MemoryAgent("src/memory/history.json")
stored_queries = memory_agent.load_entries(ticker)

### 4. Real-Time News Retrieval

**Capability**: Connects to financial news APIs to collect and filter relevant articles in real-time, ensuring responses are based on current market information.

In [27]:
news_retrieval_agent = NewsRetrievalAgent()

limit = 3
days_back = 7

json_data = news_retrieval_agent.get_news_json(company_names=companies, limit_per_source=limit, days_back=days_back)
news = [article["summary"] for article in json_data['data'][ticker]['articles']]

if 'data' in json_data and ticker in json_data['data'] and 'articles' in json_data['data'][ticker]:
    news = [article["summary"] for article in json_data['data'][ticker]['articles']]
else:
    news = []

### 5. Sentiment Analysis Integration

**Analysis**: Performs sentiment scoring on retrieved content (positive, negative, neutral) to determine market implications of each event.

**Output**: Provides an overall sentiment label that guides the specialist agent's response tone and content.

In [29]:
sentiment_agent = SentimentAnalysisAgent()
news_with_sentiment = []

# Extraer artículos correctamente
articles = json_data['data'][ticker]['articles']

for article in articles:
    sentiment = sentiment_agent.predict_one(article["summary"])
    news_with_sentiment.append({
        "summary": article["summary"],
        "sentiment": sentiment.label,
        "sentiment_score": sentiment.score,
        "explanation": sentiment.explanation
    })

overall_sentiment = "neutral"
positive_scores = [n["sentiment_score"] for n in news_with_sentiment if n["sentiment"] == "positive"]
negative_scores = [n["sentiment_score"] for n in news_with_sentiment if n["sentiment"] == "negative"]

if positive_scores and max(positive_scores) > 0.7:
    overall_sentiment = "positive"
elif negative_scores and max(negative_scores) > 0.7:
    overall_sentiment = "negative"

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### 6. Intelligent Topic Routing

**Routing Logic**: Analyzes the user query to determine the most appropriate domain specialist, ensuring expert-level responses for different financial topics (corporate_business, market_analysis, earnings, etc.).

In [30]:
topic_classifier = TopicClassifier()
topic = topic_classifier.classify(query)

### 7. Domain Specialist Response Generation

**Specialization**: Routes the query to a fine-tuned domain-specific model that generates human-readable answers with appropriate context, explanations, and financial insights.

In [31]:
specialist = Specialist(topic)
answer, prompt = specialist.respond(query=query, news_summaries=news, past_queries="", SA_label=overall_sentiment)

Loading model for topic: corporate_business from ../agents/specialized_agents/corporate_business-generator


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../agents/specialized_agents/corporate_business-generator'. Use `repo_type` argument if needed.

### 8. Evaluation & Continuous Improvement

In [ ]:
feedback = ""
# answer, prompt = specialist.respond(query=query, news_summaries=news, past_queries=stored_queries, feedback=feedback)

### 9. Knowledge Persistence & Storage

**Memory Management**: Stores all interactions with timestamps and context, building a comprehensive knowledge base for future reference and contextual understanding.

In [ ]:
entry = {
    "question": query,
    "articles": news,
    "SA": overall_sentiment,
    "feedback": feedback,
    "answer": answer
}
memory_agent.save_entry(entry, ticker)

## System Workflow Summary

1. Input: User submits financial query

2. Processing: System identifies company and retrieves historical context

3. Data Collection: Real-time news retrieval from financial APIs

4. Analysis: Sentiment scoring and topic classification

5. Routing: Query directed to appropriate domain specialist

6. Generation: Context-aware, expert response creation

7. Evaluation: Quality assessment and feedback incorporation

8. Storage: Knowledge persistence for future interactions

This pipeline demonstrates a sophisticated multi-agent architecture that transforms raw financial data into intelligent, conversational insights for end-users.